# Real-Time Face Recognition using OpenCV

### Objective:
To understand how to implement simple real-time face recognition using the Haar Cascade classifier and LBPH (Local Binary Patterns Histograms) face recognizer in OpenCV. This exercise will guide students through the process of preparing training data, training the recognizer, and performing real-time face recognition with a webcam.

### Prerequisites:
- Installed OpenCV library (opencv-contrib-python).

## Step 1: Import the Necessary Libraries

First, we need to import the OpenCV library and other necessary libraries.

In [10]:
import cv2
import numpy as np
import os

## Step 2: Load the Haar Cascade Classifier

The Haar Cascade classifier is a pre-trained model for face detection provided by OpenCV. We load the classifier from the pre-defined XML file.

In [11]:
# Load the Haar Cascade XML file
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

## Step 3: Prepare the Training Data

We prepare the training data by loading images from `known_faces` directory, converting them to grayscale, detecting faces, and extracting the face regions. Each image file is named after the person in the image.

In [12]:
# Path to the directory containing images of known faces
known_faces_dir = 'known_faces'

def prepare_training_data(data_folder_path):
    faces = []
    labels = []
    subjects = []

    # Iterate through each image file in the directory
    for filename in os.listdir(data_folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Load the image
            image_path = os.path.join(data_folder_path, filename)
            image = cv2.imread(image_path)
            
            # Convert the image to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Detect face in the image
            face = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            
            if len(face) != 0:
                (x, y, w, h) = face[0]

                # Apply offset to crop the face with a margin
                offset = 10  # You can adjust this value as needed
                x = max(x - offset, 0)
                y = max(y - offset, 0)
                w = min(w + 2 * offset, gray.shape[1] - x)
                h = min(h + 2 * offset, gray.shape[0] - y)

                face_region = gray[y:y+h, x:x+w]
                faces.append(face_region)
                labels.append(len(subjects))  # Label as the index of the person in the subjects list
                subjects.append(os.path.splitext(filename)[0])  # Add the name (filename without extension)

    return faces, labels, subjects

# Prepare training data
faces, labels, subjects = prepare_training_data(known_faces_dir)

## Step 4: Train the LBPH Face Recognizer

We create and train the LBPH face recognizer using the prepared faces and labels.

In [13]:
# Create and train the LBPH face recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

## Step 5: Initialize the Webcam

We open a connection to the webcam using OpenCV's `VideoCapture` method. The parameter `0` indicates that we want to use the default webcam.

In [14]:
# Open a connection to the webcam
cap = cv2.VideoCapture(0)

## Step 6: Capture Frames in a Loop

- We continuously capture frames from the webcam in a loop until the user decides to exit.
- Face detection works better on grayscale images, so we convert the captured frame from BGR to grayscale.
- Using the Haar Cascade classifier, we detect faces in the grayscale frame. The `detectMultiScale` method returns the coordinates and dimensions of the bounding boxes around detected faces.
- For each detected face, we predict the label and confidence using the LBPH face recognizer. We then draw a custom bounding box and display the name of the recognized face.
- We display the frame with the recognized faces and the custom bounding boxes in a window named 'Face Detection'.
- We break the loop and stop the webcam feed when the user presses the `q` key on the keyboard.
- After breaking the loop, we release the webcam and close all OpenCV windows.

In [15]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break

    # Convert the frame to grayscale (Haar Cascade works with grayscale images)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using Haar Cascade
    faces_detected = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces_detected:
        # Extract the region of interest (ROI) of the face in the grayscale frame
        roi_gray = gray_frame[y:y+h, x:x+w]
        
        # Predict the label (person) and confidence (how sure the recognizer is) for the detected face
        label, confidence = face_recognizer.predict(roi_gray)
        
        # Determine the name of the recognized person based on the label
        if confidence < 80:
            label_text = subjects[label]
            color = (0, 255, 0)  # Green color for known faces
        else:
            label_text = "Unknown"
            color = (0, 0, 255)  # Red color for unknown faces
        
        # Draw capture icon style bounding box around detected faces
        line_length = 30  # Length of the corner lines
        thickness = 2  # Thickness of the lines
        
        # Top-left corner
        cv2.line(frame, (x, y), (x + line_length, y), color, thickness)
        cv2.line(frame, (x, y), (x, y + line_length), color, thickness)
        
        # Top-right corner
        cv2.line(frame, (x + w, y), (x + w - line_length, y), color, thickness)
        cv2.line(frame, (x + w, y), (x + w, y + line_length), color, thickness)
        
        # Bottom-left corner
        cv2.line(frame, (x, y + h), (x + line_length, y + h), color, thickness)
        cv2.line(frame, (x, y + h), (x, y + h - line_length), color, thickness)
        
        # Bottom-right corner
        cv2.line(frame, (x + w, y + h), (x + w - line_length, y + h), color, thickness)
        cv2.line(frame, (x + w, y + h), (x + w, y + h - line_length), color, thickness)
        
        # Display the name of the detected face
        cv2.putText(frame, label_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness)

        # Display the confidence value below the name
        cv2.putText(frame, f'Confidence: {confidence:.2f}', (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

# When everything done, release the capture
cv2.destroyAllWindows()
cv2.waitKey(1)

-1